In [10]:
import xarray as xr
import torch

In [2]:
ds = xr.open_dataset("/glade/derecho/scratch/dkimpara/goes-cloud-dataset/goes_10km.zarr")

/glade/derecho/scratch/dkimpara/tmp/ipykernel_9113/4262793142.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_dataset("/glade/derecho/scratch/dkimpara/goes-cloud-dataset/goes_10km.zarr")


In [16]:
ds.isel(t=[0,1]).BT_or_R.shape

(6, 2, 1003, 923)

In [3]:
ref_array = ds.isel(t=0)["BT_or_R"].copy()

In [4]:
ref_array

<xarray.DataArray 'BT_or_R' (channel: 6, latitude: 1003, longitude: 923)> Size: 44MB
[5554614 values with dtype=float64]
Coordinates:
  * channel    (channel) int64 48B 4 7 8 9 10 13
  * latitude   (latitude) float64 8kB -50.1 -50.0 -49.9 -49.8 ... 49.9 50.0 50.1
  * longitude  (longitude) float64 7kB -121.1 -121.0 -120.9 ... -29.0 -28.9
    t          datetime64[ns] 8B 2017-12-17T00:06:00.142776256

In [11]:
y_pred = torch.load("/glade/u/home/dkimpara/miles-credit/pred.pt")

In [12]:
y_pred.shape

torch.Size([2, 6, 1, 1024, 960])

In [13]:
y_pred_member = y_pred[:1]

In [14]:
y_pred_member.shape

torch.Size([1, 6, 1, 1024, 960])

In [31]:
y_pred_member = y_pred_member.squeeze(2)

padded_lat = np.pad(ref_array.latitude, 
                    (11,10), 
                    mode="linear_ramp", 
                    end_values=(-50.1 - 11* 0.1, 50.1 + 10*.1))
padded_lon = np.pad(ref_array.longitude, 
                    (19,18), 
                    mode="linear_ramp", 
                    end_values=(-121.1 - 19 * 0.1, -28.9 + 18*.1))

da = xr.DataArray(y_pred_member, # 1, c, t, lat lon
                  dims=[ "t", "channel", "latitude", "longitude"],
                          coords={
                              "t": [1],
                              "channel": ref_array.channel,
                              "latitude": padded_lat,
                              "longitude": padded_lon,
                          },
                 )

In [ ]:
def train_one_epoch(epoch_index, tb_writer):

    for i, data in enumerate(training_loader):
        batch, labels = data
        
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(batch)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()
